In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#! SETUP 1
import sys, os
_snlp_book_dir = "../../../../"
sys.path.append(_snlp_book_dir) 
import math
from glob import glob
from os.path import isfile, join
from statnlpbook.vocab import Vocab
from statnlpbook.scienceie import calculateMeasures
import shutil
import string

In [3]:
### General modules
import pprint
from gensim.models import fasttext
from gensim.models import KeyedVectors
from copy import deepcopy
from collections import Counter
import numpy as np
import pandas as pd

### Our own modules
from Extra_files.modules.load_and_save import load_scienceie, save_to_ann, reformat_to_save
from Extra_files.modules.rules import _add_rules
from Extra_files.modules.error_analysis import plot_confusion_matrix, get_dataframe
from Extra_files.modules.DataPreparation import entityLocator, addPOStoDic, addLemmatoDic, inputPair, WordEmbedder
from Extra_files.modules.MasterVocab import MasterVocab
from Extra_files.modules.scaling import downscale
from Extra_files.models.dummy_model import _sebastians_dummy_model
from Extra_files.modules.Outputter import checkLegality, mostProbableYetLegal
from Extra_files.modules.CalcMeasuresTMP import calcMeasures, normaliseAnn

pp = pprint.PrettyPrinter(indent=1,depth = None, compact = True).pprint

In [4]:
from pickle import load, dump

# data_m_XY = dataX_Y_format(train_data,indices = True) 
# data_m_XY_dev = dataX_Y_format(dev_data,indices = True)
# dump(data_m_XY, open('Extra_files/resources/train_ind.pkl', 'wb'))

data_m_XY = load(open('Extra_files/resources/train_ind.pkl', 'rb'))
data_m_XY_dev = load(open('Extra_files/resources/dev_ind.pkl', 'rb'))
dev_data = load_scienceie(datadir=join(_snlp_book_dir, "data", "scienceie", "dev"))
if False:
    print(data_m_XY['metadata'][0])
    pp(data_m_XY['data_X'][0])
    print(data_m_XY['data_Y'][0])
    
data = dev_data[list(dev_data.keys())[10]] # this one has many relations

Number of entities removed due to overlap: 269 out of 1330
Number of entities not identified in text: 1 out of 1061
Number of relations lost due to overlap: 10 out of 168
Removed references.
Concatenated 'i.e.' and 'e.g.'.


In [5]:
pp(data['relations'])

[('*', 'Synonym', 'T4', 'T3'), ('*', 'Synonym', 'T6', 'T5'),
 ('R1', 'Hyponym', 'T15', 'T31'), ('R2', 'Hyponym', 'T16', 'T31'),
 ('R3', 'Hyponym', 'T17', 'T31'), ('R4', 'Hyponym', 'T18', 'T31'),
 ('R5', 'Hyponym', 'T19', 'T31')]


In [6]:
if False:
    predictions = np.array([[0.4, 0.7, 0.3, 0.1], [0.8, 0.2, 0.5, 0.1], [0.5, 0.3, 0.6, 0.2], [0.9, 0.1, 0.0, 0.0]])
    #mostProbableYetLegal(predictions)

    # rank by highest probability
    probs = np.max(predictions, axis = 1)
    highestorder = np.argsort(probs)[::-1] #flip the order
    print(highestorder)
    # sort predictions
    pred_sorted = predictions[highestorder]
    legalpreds = []
    checkpreds = []
    for pred in pred_sorted:
        print(legalpreds)
        checkpreds = deepcopy(legalpreds)
        checkpreds.append(pred)
        print(legalpreds)
        # append if legal
        if checkLegality(checkpreds):
            legalpreds.append(pred)
        #end-if
    #end-for
    print(legalpreds)

    print("the function")
    mostProbableYetLegal(predictions)

In [10]:
targetann = open("fakeANN/fakePred/example.ann", "r")
pp(normaliseAnn(targetann, ""))
targetann.close()

(['T1\tTask 910 937\tpredict corrosion behaviour',
  'T2\tProcess 846 891\tVolta potential differences in microstructure',
  'T3\tProcess 12 27\tVolta potential', 'T4\tProcess 29 30\tΨ',
  'T5\tProcess 39 59\tpotential difference',
  'T6\tProcess 12 27\tVolta potential',
  'T7\tProcess 204 240\telectrochemical behaviour of a metal',
  'T8\tProcess 254 292\tscanning Kelvin probe force microscopy',
  'T9\tProcess 294 299\tSKPFM', 'T10\tTask 318 340\tdetection of local EWF',
  'T11\tTask 378 405\tVolta potential differences', 'T12\tTask 407 409\tΔΨ',
  'T13\tMaterial 422 449\tatomic force microscopy tip',
  'T14\tMaterial 459 468\tPt coated', 'T15\tMaterial 478 491\tmetal surface',
  'T16\tProcess 530 535\tSKPFM', 'T17\tProcess 621 636\tVolta potential',
  'T18\tProcess 657 684\tproperty of a metal surface',
  'T19\tProcess 168 188\tmeasureable quantity',
  'T20\tProcess 12 27\tVolta potential',
  'T21\tTask 704 740\tunderstand electrochemical processes',
  'T22\tProcess 969 1018\tlarger 

In [22]:
calcMeasures("fakeANN/fakeGold/", "fakeANN/fakePred/")

           precision   recall f1-score  support

    Process     1.00     1.00     1.00       16
   Material     1.00     1.00     1.00        7
       Task     1.00     1.00     1.00        5
 Synonym-of     1.00     1.00     1.00        3
 Hyponym-of     0.33     1.00     0.50        2

avg / total     0.89     1.00     0.94       33




{'Hyponym-of': {'f1-score': 0.5,
  'precision': 0.3333333333333333,
  'recall': 1.0,
  'support': 2},
 'Material': {'f1-score': 1.0, 'precision': 1.0, 'recall': 1.0, 'support': 7},
 'Process': {'f1-score': 1.0, 'precision': 1.0, 'recall': 1.0, 'support': 16},
 'Synonym-of': {'f1-score': 1.0,
  'precision': 1.0,
  'recall': 1.0,
  'support': 3},
 'Task': {'f1-score': 1.0, 'precision': 1.0, 'recall': 1.0, 'support': 5},
 'overall': {'f1-score': 0.9428571428571428,
  'precision': 0.8918918918918919,
  'recall': 1.0,
  'support': 33}}

**What works and when:**<br>
[ ] Number of entities is below 2<br>
[ ] Number of entities is below 2 and includes relations<br>
[ ] Number of entities is below 2 and relations reference non-existing entities<br>
[ ] Relations reference non-existing entities<br>
[ ] Relation between the same entity: Hyponym-of Arg1:T1 Arg2:T1<br>
[ ] Relation between the same entity: Synonym-of T1 T1<br>
[ ] No ANN file (shouldn't happen but sure)<br>
[ ] Circular relations: Hyponym-of Arg1:T1 Arg2:T2 & Hyponym-of Arg1:T2 Arg2:T1
